In [2]:
# Load Libraries
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
# Reshape data to add a channel dimension
x_train = x_train.reshape((-1,28,28,1))
x_test = x_test.reshape((-1,28,28,1))

## 1. Model 1

In [13]:
build_model_1 = keras.Sequential([
    layers.Conv2D(32, (3,3), strides=(2,2), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2), strides=(1,1)),
    layers.Conv2D(16, (4,4), strides=(2,2), activation='relu'),
    layers.MaxPooling2D((4,4), strides=(2,2)),
    layers.Flatten(),
    layers.Dense(8, activation='relu'),
    layers.Dense(10, activation='softmax')
])

learning_rate = 0.01

optimizer = Adam(learning_rate=learning_rate)

build_model_1.compile(optimizer = optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy']
)

build_model_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 13, 13, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 5, 5, 16)          8208      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 1, 16)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 8)                

In [14]:
model1_accuracies = []

for i in range(1):
    
    print(f"Training Model1 {i+1}...")
    build_model_1.fit(x_train, y_train, epochs=5, batch_size = 128)
    
    print(f"Evaluating Model1 {i+1}...")
    test_loss_m1, test_accuracy_m1 = build_model_1.evaluate(x_test, y_test, verbose=2)
    model1_accuracies.append(test_accuracy_m1)

mean_test_accuracy_m1 = np.mean(model1_accuracies)
print("Mean test accuracy of Model 1: ", mean_test_accuracy_m1)

Training Model1 1...
Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 0.9783 - accuracy: 0.6521
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.4294 - accuracy: 0.8680
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.3030 - accuracy: 0.9076
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2513 - accuracy: 0.9230
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2240 - accuracy: 0.9322
Evaluating Model1 1...
313/313 - 1s - loss: 0.2083 - accuracy: 0.9336 - 635ms/epoch - 2ms/step
Mean test accuracy of Model 1:  0.9336000084877014


In [15]:
# Extract the features from second last fully connected layer (having 8 neurons)

feature_extractor_m1 = keras.Model(
    inputs=build_model_1.inputs,
    outputs = build_model_1.layers[-2].output)

#predict the feature train and test
train_features_m1 = feature_extractor_m1.predict(x_train)
test_features_m1 = feature_extractor_m1.predict(x_test)

# Model features using Random Forest classifier
rf_classifier_m1 = RandomForestClassifier(n_estimators = 100, random_state=42)
rf_classifier_m1.fit(train_features_m1, y_train)

# Predict using the Random Forest classifier
predictions_m1 = rf_classifier_m1.predict(test_features_m1)

# Calculate accuracy
accuracy_m1 = accuracy_score(y_test, predictions_m1)
print("Accuracy of Random Forest classifier on extracted features model 1:", accuracy_m1)


313/313 [==============================] - 0s 1ms/step
Accuracy of Random Forest classifier on extracted features model 1: 0.9401


## Model 2

In [16]:
build_model_2 = keras.Sequential([
    layers.Conv2D(32, (3,3), strides=(2,2), activation='relu', input_shape=(28,28,1)),
    layers.AveragePooling2D((2,2), strides=(1,1)),
    layers.Conv2D(16, (4,4), strides=(2,2), activation='relu'),
    layers.AveragePooling2D((4,4), strides=(2,2)),
    layers.Flatten(),
    layers.Dense(8, activation='relu'),
    layers.Dense(10, activation='softmax')
])

learning_rate = 0.01

optimizer = Adam(learning_rate=learning_rate)

build_model_2.compile(optimizer = optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy']
                      )

build_model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 13, 13, 32)        320       
                                                                 
 average_pooling2d_2 (Averag  (None, 12, 12, 32)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_7 (Conv2D)           (None, 5, 5, 16)          8208      
                                                                 
 average_pooling2d_3 (Averag  (None, 1, 1, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten_3 (Flatten)         (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 8)                

In [17]:
model2_accuracies = []

for i in range(1):
    
    print(f"Training Model2 {i+1}...")
    build_model_2.fit(x_train, y_train, epochs=5, batch_size = 128)
    
    print(f"Evaluating Model2 {i+1}...")
    test_loss_m2, test_accuracy_m2 = build_model_2.evaluate(x_test, y_test, verbose=2)
    model2_accuracies.append(test_accuracy_m2)

mean_test_accuracy_m2 = np.mean(model2_accuracies)
print("Mean test accuracy of Model 2: ", mean_test_accuracy_m2)

Training Model2 1...
Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 1.3158 - accuracy: 0.5099
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.6422 - accuracy: 0.7858
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.4692 - accuracy: 0.8511
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.4024 - accuracy: 0.8742
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.3555 - accuracy: 0.8912
Evaluating Model2 1...
313/313 - 1s - loss: 0.3185 - accuracy: 0.9007 - 669ms/epoch - 2ms/step
Mean test accuracy of Model 2:  0.9006999731063843
